In [25]:
import pandas as pd

In [26]:
# 定义相对路径
file_data_dictionary = 'data/data_dictionary.csv'
file_summerOly_athletes = 'data/summerOly_athletes.csv'
file_summerOly_hosts = 'data/summerOly_hosts.csv'
file_summerOly_medal_counts = 'data/summerOly_medal_counts.csv'
file_summerOly_programs = 'data/summerOly_programs.csv'
file_Countries = 'ProcessedData/Countries.csv'
file_Athletes = 'ProcessedData/Athletes.csv'
file_Medals = 'ProcessedData/Medals.csv'

# 读取表格
df_data_dictionary = pd.read_csv(
    file_data_dictionary,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)

df_summerOly_athletes = pd.read_csv(
    file_summerOly_athletes,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
# 筛选并修改 'NOC' 列中 LBA 为 LBI
df_summerOly_athletes.loc[df_summerOly_athletes['NOC'] == 'LBA', 'NOC'] = 'LBI'
# 筛选并修改 'NOC' 列中 ROC 为 RUS
df_summerOly_athletes.loc[df_summerOly_athletes['NOC'] == 'ROC', 'NOC'] = 'RUS'
# 创建映射字典
replacement_map = {
    'North Macedonia': 'Macedonia',
    'Taiwan': 'Chinese Taipei',
    'FR Yugoslavia': 'Yugoslavia',
    'Formosa': 'Chinese Taipei',
    'Ceylon': 'Sri Lanka',
    'British West Indies': 'Great Britain'
}
# 使用 replace 方法替换
df_summerOly_athletes['Team'] = df_summerOly_athletes['Team'].replace(replacement_map)

df_summerOly_hosts = pd.read_csv(
    file_summerOly_hosts,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)

df_summerOly_medal_counts = pd.read_csv(
    file_summerOly_medal_counts,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
df_summerOly_medal_counts['NOC'] = df_summerOly_medal_counts['NOC'].replace(replacement_map)

df_summerOly_programs = pd.read_csv(
    file_summerOly_programs,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)

Countries = pd.read_csv(file_Countries)

In [27]:
Time_Medals = pd.read_csv(file_Medals)
# 计算过去一届奥运会的金牌数和总奖牌数
Time_Medals.sort_values(by=['NOC', 'Year'])

,Rank,NOC,Gold,Silver,Bronze,Total,Year
1068,82,AFG,0,0,1,1,2008
1152,79,AFG,0,0,1,1,2012
674,36,AHO,0,1,0,1,1988
1417,80,ALB,0,0,2,2,2024
633,42,ALG,0,0,2,2,1984
...,...,...,...,...,...,...,...
820,61,ZAM,0,1,0,1,1996
1428,84,ZAM,0,0,1,1,2024
580,23,ZIM,1,0,0,1,1980
961,49,ZIM,1,1,1,3,2004


In [28]:
Time_Medals = pd.read_csv(file_Medals)
# 计算过去一届奥运会的金牌数和总奖牌数
Time_Medals['Past_Gold'] = Time_Medals.groupby('NOC')['Gold'].shift(1)
Time_Medals['Past_Silver'] = Time_Medals.groupby('NOC')['Silver'].shift(1)
Time_Medals['Past_Bronze'] = Time_Medals.groupby('NOC')['Bronze'].shift(1)
Time_Medals['Past_Total'] = Time_Medals.groupby('NOC')['Total'].shift(1)
Time_Medals.sort_values(by=['NOC', 'Year'])
Time_Medals.to_csv('ProcessedData/timeMedalsByCountry.csv')

In [29]:
Events_count = df_summerOly_athletes[['NOC', 'Year', 'Event']]
Events_count = Events_count.groupby(['NOC', 'Year'])['Event'].count().reset_index(name='Events_Count')
Events_count

,NOC,Year,Events_Count
0,AFG,1936,16
1,AFG,1948,25
2,AFG,1956,12
3,AFG,1960,16
4,AFG,1964,8
...,...,...,...
3217,ZIM,2008,16
3218,ZIM,2012,9
3219,ZIM,2016,31
3220,ZIM,2020,5


In [30]:
Sports_count = df_summerOly_athletes[['NOC', 'Year', 'Sport']]
Sports_count = Sports_count.groupby(['NOC', 'Year'])['Sport'].count().reset_index(name='Sports_Count')
Sports_count

,NOC,Year,Sports_Count
0,AFG,1936,16
1,AFG,1948,25
2,AFG,1956,12
3,AFG,1960,16
4,AFG,1964,8
...,...,...,...
3217,ZIM,2008,16
3218,ZIM,2012,9
3219,ZIM,2016,31
3220,ZIM,2020,5


In [31]:
data = pd.merge(Time_Medals, Events_count, on=['NOC', 'Year'], how='outer', left_index=False, right_index=False).fillna(0)
data = pd.merge(data, Sports_count, on=['NOC', 'Year'], how='left', left_index=False, right_index=False)
data = data.drop(columns=['Rank'])
data = data.astype(str).drop_duplicates(keep='first')
data.sort_values(by=['NOC', 'Year'])

,NOC,Gold,Silver,Bronze,Total,Year,Past_Gold,Past_Silver,Past_Bronze,Past_Total,Events_Count,Sports_Count
0,AFG,0.0,0.0,0.0,0.0,1936,0.0,0.0,0.0,0.0,16.0,16.0
1,AFG,0.0,0.0,0.0,0.0,1948,0.0,0.0,0.0,0.0,25.0,25.0
2,AFG,0.0,0.0,0.0,0.0,1956,0.0,0.0,0.0,0.0,12.0,12.0
3,AFG,0.0,0.0,0.0,0.0,1960,0.0,0.0,0.0,0.0,16.0,16.0
4,AFG,0.0,0.0,0.0,0.0,1964,0.0,0.0,0.0,0.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3223,ZIM,1.0,3.0,0.0,4.0,2008,1.0,1.0,1.0,3.0,16.0,16.0
3224,ZIM,0.0,0.0,0.0,0.0,2012,0.0,0.0,0.0,0.0,9.0,9.0
3225,ZIM,0.0,0.0,0.0,0.0,2016,0.0,0.0,0.0,0.0,31.0,31.0
3226,ZIM,0.0,0.0,0.0,0.0,2020,0.0,0.0,0.0,0.0,5.0,5.0


In [32]:
df_summerOly_hosts['Country_Name'] = df_summerOly_hosts['Host'].str.split(',').str[-1].str.strip()
host_Country = df_summerOly_hosts['Country_Name']
host_Country = pd.merge(host_Country, Countries, on='Country_Name', how='left')

host_Country['Host_Years'] = host_Country['Host_Years'].apply(lambda x: eval(x) if isinstance(x, str) else x)
# 然后将每个年份拆分成不同的行
df_host_expanded = host_Country.explode('Host_Years')
df_host_expanded = df_host_expanded[['NOC', 'Host_Years']].rename(columns={'Host_Years': 'Year'})
df_host_expanded['isHoster'] = df_host_expanded['NOC'].apply(lambda x: 1 if x != 'NaN' else 0)
# df_host_expanded

host_Country_data = pd.merge(data, df_host_expanded, on=['NOC', 'Year'], how='left', left_index=False,
                             right_index=False)
host_Country_data['isHoster'] = host_Country_data['isHoster'].fillna(0)
host_Country_data.sort_values(by=['NOC', 'Year']).drop_duplicates(keep='first')
host_Country_data.to_csv('ProcessedData/data4Supervised.csv')
host_Country_data.sort_values(by=['NOC', 'Year'])
print(host_Country_data.head(1))


   NOC Gold Silver Bronze Total  Year Past_Gold Past_Silver Past_Bronze  \
0  AFG  0.0    0.0    0.0   0.0  1936       0.0         0.0         0.0   

  Past_Total Events_Count Sports_Count  isHoster  
0        0.0         16.0         16.0       0.0  
